In [1]:
import numpy as np
import os
import pandas as pd
from numpy.ma.extras import average
"""_vehicle_output 파일에서 데이터 가공 (차종별 평균, 가중 평균속도)"""
# 오류 날 시 2차로, 3차로 차로수 별 컬럼 확인
folder_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\시뮬레이션 분석자료 모음\자료정리(3차로_점선_지상부 추가)_250728\시뮬레이션 분석 자료_V2.9\차종별"

vehicle_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)
print("vehicle_files : ", vehicle_files)
for n in range(len(vehicle_files)):
    df = pd.read_excel(os.path.join(folder_path, vehicle_files[n]))
    df["NewMeasurem"] = df["Measurem."] % 1000

    vehicle_type_list = [100, 300, 630, 640, 650]
    results = []
    for i in range(len(vehicle_type_list)):
        filtered_df = df[df["Vehicle type"] == vehicle_type_list[i]]
        # 피벗 테이블 생성
        old_df = filtered_df.pivot_table(
            index="time",
            columns=["NewMeasurem", "Measurem."],
            values="v[km/h]",
            aggfunc="mean"
        )
        # 결과를 저장할 딕셔너리
        combined_cols = {}

        # level=0("NewMeasurem") 별로 그룹핑 후 2개 열의 데이터를 이어 붙인 후 새로운 컬럼 생성
        for measurem_group, cols in old_df.groupby(axis=1, level=0):
            if len(cols.columns) == 3: # 차로수
                col1, col2, col3 = cols.columns # 2차로면 col1, col2 / 3차로면 col1, col2, col3
                new_col_name = f'{col1[1]}'
                # 1번째 열과 2번째 열을 하나로 합친다.
                # 2차로 : cols[col1], cols[col2] / 3차로 : cols[col1], cols[col2], cols[col3]
                combined_series = pd.concat([cols[col1], cols[col2], cols[col3]], ignore_index=True)
                combined_cols[new_col_name] = combined_series
        combined_df = pd.DataFrame(combined_cols)

        # 평균속도
        avg_speed = combined_df.mean(axis=0)
        # 표준편차
        std = combined_df.std(ddof=0)

        # 변동계수
        vrt_num = std / avg_speed
        # 가중치
        add_per =  1/vrt_num

        for j in range(len(avg_speed)):
            results.append({
                "Vehicle type": vehicle_type_list[i],
                "NewMeasurem": int(list(avg_speed.index)[j]) % 1000,
                "AverageSpeed": avg_speed.iloc[j],
                "Weight (가중치)": add_per.iloc[j]
            })
    weights_df = pd.DataFrame(results)
    #print(weights_df[(weights_df["Vehicle type"] == 630) & (weights_df["NewMeasurem"] == 2)])
    ####### 가중 평균속도 계산 #######

    # 평균속도 * 가중치
    weights_df["Weighted Value"] = weights_df["AverageSpeed"] * weights_df["Weight (가중치)"]

    # NewMeasurem별로 그룹화 및 합 계산
    sum_weighted = weights_df.groupby("NewMeasurem")["Weighted Value"].sum()
    sum_weights = weights_df.groupby("NewMeasurem")["Weight (가중치)"].sum()

    # 가중 평균 속도 계산
    final_df = (sum_weighted / sum_weights).reset_index(name="Weighted Avg Speed")
    #print(final_df)
    final_df = final_df.set_index("NewMeasurem").T
    final_df.index = ["가중평균속도"]  # 이게 첫 번째 행(인덱스)로 들어감


    # 차종별 평균속도 계산 (NewMeasurem 기준)
    vehicle_avg = weights_df.groupby(["NewMeasurem", "Vehicle type"])["AverageSpeed"].mean().unstack()
    vehicle_avg = vehicle_avg.reindex(columns=vehicle_type_list)  # 차종 순서 고정
    vehicle_avg = vehicle_avg
    vehicle_avg.index.name = "NewMeasurem"
    vehicle_avg.columns = [f"차종{col}" for col in vehicle_avg.columns]

    # 가중평균속도와 결합
    final_df = pd.concat([vehicle_avg.T, final_df])
    excel_dir = os.path.join(folder_path, "weights_output")
    if not os.path.exists(excel_dir):
        os.makedirs(excel_dir)
    last_file = xlsx_files = sorted([f for f in os.listdir(excel_dir) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not xlsx_files: # 폴더에 파일이 존재하지 않을 때의 시작 번호
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0]) + 1  # 가장 마지막 파일 번호 + 1
    excel_path = os.path.join(excel_dir, f"{last_file}_weights_output.xlsx")
    final_df.to_excel(excel_path, index=True)
    print(f"생성파일: {excel_path}")


vehicle_files :  ['1_vehicle_output.xlsx', '2_vehicle_output.xlsx', '3_vehicle_output.xlsx', '4_vehicle_output.xlsx', '5_vehicle_output.xlsx', '6_vehicle_output.xlsx', '7_vehicle_output.xlsx', '8_vehicle_output.xlsx', '9_vehicle_output.xlsx', '10_vehicle_output.xlsx', '11_vehicle_output.xlsx', '12_vehicle_output.xlsx', '13_vehicle_output.xlsx', '14_vehicle_output.xlsx', '15_vehicle_output.xlsx', '16_vehicle_output.xlsx', '17_vehicle_output.xlsx', '18_vehicle_output.xlsx', '19_vehicle_output.xlsx', '20_vehicle_output.xlsx', '21_vehicle_output.xlsx', '22_vehicle_output.xlsx', '23_vehicle_output.xlsx', '24_vehicle_output.xlsx', '25_vehicle_output.xlsx', '26_vehicle_output.xlsx', '27_vehicle_output.xlsx', '28_vehicle_output.xlsx', '29_vehicle_output.xlsx', '30_vehicle_output.xlsx', '31_vehicle_output.xlsx', '32_vehicle_output.xlsx', '33_vehicle_output.xlsx', '34_vehicle_output.xlsx', '35_vehicle_output.xlsx', '36_vehicle_output.xlsx', '37_vehicle_output.xlsx', '38_vehicle_output.xlsx', '39